### Notes

* changed shape[1] from 4 to 6

In [1]:
%matplotlib inline

In [2]:
import sys
import numpy as np
import tensorflow as tf

from batcher import Batcher
from metrics import acc012

In [3]:
X_train = np.load('../../data/bin/train/deal.npy')
y_train = np.load('../../data/bin/train/tricks_notrump.npy')

X_val = np.load('../../data/bin/val/deal.npy')
y_val = np.load('../../data/bin/val/tricks_notrump.npy')

n_examples = X_train.shape[0]

n_h = X_train.shape[1]
n_w = X_train.shape[2]
n_c = X_train.shape[3]

batch_size = 64
n_iterations = 2000000
display_step = 1000

learning_rate = 0.001

n_hidden_units = 128

l2_reg = 0.05

In [4]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((10800000, 4, 13, 4), (100000, 4, 13, 4), (10800000, 1), (100000, 1))

In [5]:
X = tf.placeholder(tf.float32, shape=[None, n_h, n_w, n_c])
Y = tf.placeholder(tf.float32, shape=[1, None])
strides = [1,1,1,1]

conv1_w = tf.get_variable('c1w', shape=[1, 6, 4, 32], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer(seed=1337))
conv1_z = tf.nn.conv2d(X, filter=conv1_w, strides=strides, padding='SAME')
conv1_a = tf.nn.relu(conv1_z)

conv2_w = tf.get_variable('c2w', shape=[1, 6, 32, 64], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer(seed=1337))
conv2_z = tf.nn.conv2d(conv1_a, filter=conv2_w, strides=strides, padding='SAME')
conv2_a = tf.nn.relu(conv2_z)

conv3_w = tf.get_variable('c3w', shape=[1, 6, 64, 128], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer(seed=1337))
conv3_z = tf.nn.conv2d(conv2_a, filter=conv3_w, strides=strides, padding='SAME')
conv3_a = tf.nn.relu(conv3_z)

conv4_w = tf.get_variable('c4w', shape=[4, 6, 128, 512], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer(seed=1337))
conv4_z = tf.nn.conv2d(conv3_a, filter=conv4_w, strides=strides, padding='VALID')
conv4_a = tf.nn.relu(conv4_z)

fc_in = tf.contrib.layers.flatten(conv4_a)
fc_w = tf.get_variable('fcw', shape=[n_hidden_units, fc_in.shape.as_list()[1]], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer(seed=1337))
fc_b = tf.Variable(np.zeros((n_hidden_units, 1)), dtype=tf.float32)
fc_z = tf.add(tf.matmul(fc_w, tf.transpose(fc_in)), fc_b)
fc_a = tf.nn.relu(fc_z)

w_out = tf.get_variable('w_out', shape=[1, n_hidden_units], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer(seed=1337))
b_out = tf.Variable(np.zeros((1, 1)), dtype=tf.float32)
pred = tf.add(tf.matmul(w_out, fc_a), b_out)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.


In [6]:
cost = tf.reduce_mean(tf.squared_difference(pred, Y))

In [7]:
optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=0.9).minimize(cost)

In [8]:
init = tf.global_variables_initializer()

In [9]:
batch = Batcher(n_examples, batch_size)
cost_train_batch = Batcher(n_examples, 10000)
cost_val_batch = Batcher(100000, 10000)

In [10]:
costs = []

with tf.Session() as sess:
    sess.run(init)
    
    for iteration in range(n_iterations):
        x_batch, y_batch = batch.next_batch([X_train, y_train])
        
        if iteration % display_step == 0:
            sys.stdout.write('*')
            x_batch_c, y_batch_c = cost_train_batch.next_batch([X_train, y_train])
            x_batch_v, y_batch_v = cost_val_batch.next_batch([X_val, y_val])
            c = sess.run(cost, feed_dict={X: x_batch_c, Y: y_batch_c.T})
            costs.append(c)
            pred_train = sess.run(pred, feed_dict={X: x_batch_c, Y: y_batch_c.T})
            pred_val = sess.run(pred, feed_dict={X: x_batch_v, Y: y_batch_v.T})
            print('it={} cost={}'.format(iteration, c))
            print(acc012(y_batch_c, pred_train.T))
            print(acc012(y_batch_v, pred_val.T))
            
        sess.run(optimizer, feed_dict={X:x_batch, Y:y_batch.T})
    
    # Testing
    graph = tf.get_default_graph()
    
    X_test = np.load('../../data/bin/test/deal.npy')
    y_test = np.load('../../data/bin/test/tricks_notrump.npy')
    
    print('\n\nTesting')
    predictions = sess.run(pred, feed_dict = {X:X_test})

    print(acc012(y_test.ravel(), predictions.round().ravel()))

*it=0 cost=43.97013854980469
(0.0163, 0.0494, 0.1066)
(0.0148, 0.0511, 0.1107)
*it=1000 cost=1.4318467378616333
(0.3394, 0.8039, 0.9589)
(0.3458, 0.804, 0.9633)
*it=2000 cost=1.363612174987793
(0.3497, 0.8141, 0.962)
(0.3486, 0.8206, 0.9662)
*it=3000 cost=1.1958221197128296
(0.3639, 0.8364, 0.9736)
(0.3761, 0.8445, 0.9741)
*it=4000 cost=1.0511482954025269
(0.3924, 0.8682, 0.9782)
(0.3937, 0.8628, 0.9777)
*it=5000 cost=1.0444302558898926
(0.3983, 0.8693, 0.9776)
(0.4097, 0.8717, 0.9784)
*it=6000 cost=0.9805476665496826
(0.4106, 0.8817, 0.9825)
(0.4039, 0.8745, 0.9811)
*it=7000 cost=1.0263358354568481
(0.4058, 0.8745, 0.98)
(0.4121, 0.8719, 0.9821)
*it=8000 cost=0.8976501822471619
(0.4273, 0.8906, 0.9851)
(0.4241, 0.8879, 0.9833)
*it=9000 cost=0.9409404397010803
(0.4341, 0.8876, 0.9823)
(0.4342, 0.8894, 0.9831)
*it=10000 cost=0.88948655128479
(0.4313, 0.8958, 0.9854)
(0.4372, 0.894, 0.9833)
*it=11000 cost=0.8680914044380188
(0.436, 0.9009, 0.9851)
(0.4498, 0.897, 0.9862)
*it=12000 cost=0

*it=99000 cost=0.39901819825172424
(0.6426, 0.9731, 0.997)
(0.6243, 0.9732, 0.9969)
*it=100000 cost=0.393031507730484
(0.6244, 0.9732, 0.997)
(0.6202, 0.9688, 0.9971)
*it=101000 cost=0.398640513420105
(0.6266, 0.9728, 0.9978)
(0.6164, 0.9693, 0.9976)
*it=102000 cost=0.40984198451042175
(0.6234, 0.9702, 0.9972)
(0.6154, 0.9694, 0.9979)
*it=103000 cost=0.4144376814365387
(0.6256, 0.9685, 0.997)
(0.6175, 0.9735, 0.9974)
*it=104000 cost=0.4039978086948395
(0.6194, 0.9712, 0.998)
(0.6019, 0.9714, 0.9964)
*it=105000 cost=0.4078216552734375
(0.6307, 0.9696, 0.9971)
(0.6257, 0.969, 0.9963)
*it=106000 cost=0.4104748070240021
(0.6163, 0.9713, 0.9965)
(0.6187, 0.9722, 0.9969)
*it=107000 cost=0.3844727873802185
(0.6368, 0.9758, 0.9976)
(0.6335, 0.9709, 0.9967)
*it=108000 cost=0.40930479764938354
(0.6185, 0.9716, 0.9961)
(0.6344, 0.9741, 0.9967)
*it=109000 cost=0.393993616104126
(0.6365, 0.972, 0.9969)
(0.626, 0.975, 0.9961)
*it=110000 cost=0.3871593773365021
(0.6454, 0.9736, 0.9966)
(0.6349, 0.971

*it=196000 cost=0.3114515542984009
(0.6878, 0.9827, 0.9982)
(0.6827, 0.9811, 0.9976)
*it=197000 cost=0.33007681369781494
(0.6739, 0.9806, 0.9983)
(0.6687, 0.9791, 0.9977)
*it=198000 cost=0.3090653419494629
(0.6863, 0.9835, 0.9991)
(0.6613, 0.9814, 0.9986)
*it=199000 cost=0.3124415874481201
(0.685, 0.9818, 0.9988)
(0.68, 0.9809, 0.9981)
*it=200000 cost=0.34847432374954224
(0.6643, 0.9791, 0.9978)
(0.6678, 0.9776, 0.9978)
*it=201000 cost=0.33688315749168396
(0.6736, 0.9802, 0.9977)
(0.6779, 0.9802, 0.9979)
*it=202000 cost=0.31873199343681335
(0.6863, 0.9816, 0.9986)
(0.6817, 0.9814, 0.9978)
*it=203000 cost=0.33377179503440857
(0.67, 0.9815, 0.9982)
(0.665, 0.9801, 0.9988)
*it=204000 cost=0.31982794404029846
(0.6781, 0.9821, 0.9986)
(0.6851, 0.9815, 0.9987)
*it=205000 cost=0.31926852464675903
(0.6758, 0.9831, 0.9982)
(0.6784, 0.981, 0.9983)
*it=206000 cost=0.30242809653282166
(0.6862, 0.9858, 0.9983)
(0.6794, 0.984, 0.9987)
*it=207000 cost=0.3342079520225525
(0.6782, 0.9808, 0.9983)
(0.68

*it=293000 cost=0.2805967628955841
(0.71, 0.9861, 0.9994)
(0.7033, 0.9845, 0.9989)
*it=294000 cost=0.2892792522907257
(0.7016, 0.986, 0.9987)
(0.6996, 0.9839, 0.9983)
*it=295000 cost=0.28133633732795715
(0.7053, 0.9855, 0.9989)
(0.6941, 0.9833, 0.9987)
*it=296000 cost=0.27626729011535645
(0.7125, 0.9874, 0.9988)
(0.7103, 0.9844, 0.9988)
*it=297000 cost=0.27354896068573
(0.7138, 0.9877, 0.9992)
(0.7031, 0.9851, 0.9989)
*it=298000 cost=0.2959413230419159
(0.6883, 0.9871, 0.9986)
(0.6903, 0.9838, 0.9982)
*it=299000 cost=0.28170260787010193
(0.7101, 0.986, 0.9987)
(0.7002, 0.985, 0.9986)
*it=300000 cost=0.27311772108078003
(0.7174, 0.9875, 0.9986)
(0.7067, 0.9863, 0.9992)
*it=301000 cost=0.2744334936141968
(0.7083, 0.9872, 0.9989)
(0.7013, 0.9856, 0.9987)
*it=302000 cost=0.27519911527633667
(0.7086, 0.9869, 0.9992)
(0.7003, 0.9842, 0.9986)
*it=303000 cost=0.27529725432395935
(0.7182, 0.9867, 0.9989)
(0.7055, 0.9854, 0.998)
*it=304000 cost=0.2900877594947815
(0.7046, 0.9859, 0.9981)
(0.7008

*it=390000 cost=0.25281330943107605
(0.7293, 0.9897, 0.999)
(0.715, 0.9878, 0.9988)
*it=391000 cost=0.25363633036613464
(0.7284, 0.9875, 0.9995)
(0.7277, 0.9892, 0.9989)
*it=392000 cost=0.2486410140991211
(0.7305, 0.9911, 0.9991)
(0.7302, 0.988, 0.9996)
*it=393000 cost=0.2548358142375946
(0.7382, 0.9887, 0.9989)
(0.7267, 0.9867, 0.9989)
*it=394000 cost=0.2509441375732422
(0.7314, 0.9902, 0.9989)
(0.7233, 0.9864, 0.9986)
*it=395000 cost=0.25127941370010376
(0.7322, 0.9879, 0.9991)
(0.731, 0.9897, 0.9987)
*it=396000 cost=0.25647079944610596
(0.7335, 0.9884, 0.999)
(0.7226, 0.9887, 0.9992)
*it=397000 cost=0.26470044255256653
(0.7196, 0.989, 0.9985)
(0.7171, 0.9844, 0.9984)
*it=398000 cost=0.2485021948814392
(0.736, 0.9884, 0.9994)
(0.7203, 0.9864, 0.9986)
*it=399000 cost=0.26128485798835754
(0.7308, 0.9886, 0.9987)
(0.7266, 0.9868, 0.9985)
*it=400000 cost=0.2574146091938019
(0.733, 0.9879, 0.9987)
(0.7287, 0.9862, 0.9985)
*it=401000 cost=0.25187572836875916
(0.7326, 0.9888, 0.9994)
(0.725

...

*it=1548000 cost=0.17425663769245148
(0.7976, 0.9961, 0.9999)
(0.7862, 0.9946, 0.9996)
*it=1549000 cost=0.16932877898216248
(0.8029, 0.9963, 0.9997)
(0.7748, 0.9929, 0.9993)
*it=1550000 cost=0.17450036108493805
(0.7969, 0.9961, 0.9998)
(0.7808, 0.9943, 0.9997)
*it=1551000 cost=0.16730202734470367
(0.8075, 0.9959, 0.9999)
(0.7823, 0.9926, 0.9989)
*it=1552000 cost=0.17448483407497406
(0.8025, 0.9949, 0.9997)
(0.7842, 0.9939, 0.9997)
*it=1553000 cost=0.16254878044128418
(0.8125, 0.997, 0.9998)
(0.788, 0.9936, 0.9998)
*it=1554000 cost=0.1694546937942505
(0.8026, 0.9954, 0.9999)
(0.7817, 0.9936, 0.9994)
*it=1555000 cost=0.16599158942699432
(0.8107, 0.9969, 0.9998)
(0.7847, 0.9943, 0.9997)
*it=1556000 cost=0.17284025251865387
(0.8052, 0.9945, 0.9996)
(0.7808, 0.9931, 0.9991)
*it=1557000 cost=0.16888701915740967
(0.8027, 0.9969, 0.9999)
(0.7778, 0.9931, 0.9995)
*it=1558000 cost=0.1659327894449234
(0.8134, 0.996, 0.9999)
(0.7788, 0.9935, 0.9998)
*it=1559000 cost=0.16938689351081848
(0.8065, 0.

*it=1644000 cost=0.16869528591632843
(0.8102, 0.9961, 0.9998)
(0.7794, 0.9951, 0.9998)
*it=1645000 cost=0.16679096221923828
(0.803, 0.9962, 0.9996)
(0.7854, 0.9933, 0.9996)
*it=1646000 cost=0.16230477392673492
(0.8091, 0.9968, 0.9999)
(0.7846, 0.9939, 0.9995)
*it=1647000 cost=0.16791903972625732
(0.8082, 0.9961, 0.9996)
(0.7849, 0.993, 0.9994)
*it=1648000 cost=0.16329365968704224
(0.808, 0.9974, 0.9998)
(0.7879, 0.9943, 0.9997)
*it=1649000 cost=0.16306999325752258
(0.8106, 0.9966, 0.9997)
(0.7869, 0.9935, 0.9997)
*it=1650000 cost=0.16905753314495087
(0.807, 0.9964, 0.9998)
(0.7823, 0.9933, 0.9999)
*it=1651000 cost=0.17168982326984406
(0.8045, 0.9955, 0.9994)
(0.785, 0.9931, 0.9993)
*it=1652000 cost=0.1656385362148285
(0.8132, 0.9952, 0.9997)
(0.7876, 0.9941, 0.9999)
*it=1653000 cost=0.16832172870635986
(0.8111, 0.9963, 0.9996)
(0.7828, 0.9932, 0.9995)
*it=1654000 cost=0.16504602134227753
(0.8055, 0.9972, 0.9997)
(0.7869, 0.9932, 0.9998)
*it=1655000 cost=0.16779816150665283
(0.8035, 0.9

*it=1740000 cost=0.16700977087020874
(0.8083, 0.9967, 0.9998)
(0.789, 0.9952, 0.9999)
*it=1741000 cost=0.164595827460289
(0.8138, 0.9968, 0.9997)
(0.781, 0.9942, 0.9996)
*it=1742000 cost=0.16355271637439728
(0.8096, 0.9971, 0.9999)
(0.7826, 0.9919, 0.9995)
*it=1743000 cost=0.16328218579292297
(0.8123, 0.9969, 0.9995)
(0.7854, 0.9939, 0.9997)
*it=1744000 cost=0.16245898604393005
(0.8121, 0.9968, 0.9999)
(0.7874, 0.9953, 0.9998)
*it=1745000 cost=0.1744159460067749
(0.7967, 0.9962, 1.0)
(0.7841, 0.9945, 0.9997)
*it=1746000 cost=0.16371914744377136
(0.8181, 0.9958, 0.9997)
(0.7841, 0.9942, 0.9993)
*it=1747000 cost=0.16988605260849
(0.8016, 0.9957, 0.9996)
(0.7714, 0.9915, 0.9993)
*it=1748000 cost=0.16359134018421173
(0.8112, 0.9962, 0.9998)
(0.783, 0.9936, 0.9998)
*it=1749000 cost=0.16607607901096344
(0.8051, 0.9964, 1.0)
(0.7848, 0.9936, 0.9993)
*it=1750000 cost=0.16181504726409912
(0.811, 0.9968, 0.9998)
(0.7857, 0.9932, 0.9996)
*it=1751000 cost=0.1619330644607544
(0.8091, 0.9973, 1.0)
(

*it=1836000 cost=0.15961730480194092
(0.8162, 0.9968, 0.9998)
(0.781, 0.9946, 0.9997)
*it=1837000 cost=0.16039811074733734
(0.8115, 0.9976, 0.9999)
(0.7876, 0.9946, 0.9995)
*it=1838000 cost=0.15829017758369446
(0.8183, 0.9964, 0.9998)
(0.7851, 0.9937, 0.9995)
*it=1839000 cost=0.1590045839548111
(0.812, 0.9974, 0.9997)
(0.7789, 0.9947, 0.9998)
*it=1840000 cost=0.16876932978630066
(0.8025, 0.9974, 0.9999)
(0.7858, 0.9953, 0.9996)
*it=1841000 cost=0.1578793227672577
(0.8177, 0.997, 1.0)
(0.7878, 0.993, 0.999)
*it=1842000 cost=0.1604233831167221
(0.8127, 0.9972, 0.9999)
(0.7893, 0.9927, 0.9996)
*it=1843000 cost=0.15682001411914825
(0.8158, 0.9973, 1.0)
(0.7874, 0.9935, 0.9996)
*it=1844000 cost=0.16074897348880768
(0.8176, 0.9964, 0.9999)
(0.7831, 0.994, 0.9998)
*it=1845000 cost=0.15794208645820618
(0.8118, 0.9968, 0.9999)
(0.7884, 0.9943, 0.9997)
*it=1846000 cost=0.1670241504907608
(0.805, 0.9965, 1.0)
(0.7855, 0.9934, 0.9995)
*it=1847000 cost=0.1625998467206955
(0.8127, 0.9968, 0.9997)
(0

*it=1932000 cost=0.15759101510047913
(0.8206, 0.9966, 1.0)
(0.7807, 0.9944, 0.9996)
*it=1933000 cost=0.15245456993579865
(0.8238, 0.9975, 1.0)
(0.7935, 0.9931, 0.9996)
*it=1934000 cost=0.15881359577178955
(0.8131, 0.9969, 0.9999)
(0.7844, 0.9956, 0.9998)
*it=1935000 cost=0.1572152078151703
(0.8167, 0.9971, 1.0)
(0.7879, 0.9931, 0.9993)
*it=1936000 cost=0.15446388721466064
(0.8245, 0.9968, 0.9999)
(0.7859, 0.9943, 0.9998)
*it=1937000 cost=0.16694939136505127
(0.8078, 0.9971, 0.9997)
(0.7842, 0.9933, 0.9995)
*it=1938000 cost=0.15926340222358704
(0.8163, 0.9966, 0.9999)
(0.7886, 0.9942, 0.9995)
*it=1939000 cost=0.16196775436401367
(0.8141, 0.9973, 0.9998)
(0.7779, 0.9918, 0.9998)
*it=1940000 cost=0.15890361368656158
(0.8174, 0.9971, 0.9999)
(0.7802, 0.993, 0.9999)
*it=1941000 cost=0.16590985655784607
(0.8082, 0.9962, 0.9997)
(0.78, 0.9934, 0.9996)
*it=1942000 cost=0.1590295433998108
(0.8087, 0.9979, 0.9998)
(0.7836, 0.9949, 0.9998)
*it=1943000 cost=0.15452253818511963
(0.8209, 0.9968, 0.9

In [11]:
import pandas as pd
path = '../../bridgent/preds/z_v3_preds.pickle'
pd.to_pickle({'preds':predictions, 'acc012':acc012(y_test.ravel(), predictions.round().ravel())}, path)

In [12]:
pd.read_pickle(path)

{'preds': array([[4.794734 , 3.4782996, 7.052602 , ..., 5.5807095, 5.9386206,
         7.9604373]], dtype=float32),
 'acc012': (0.78035, 0.99313, 0.99941)}